# Using XGBoost in Python Tutorial 

Discover the power of XGBoost, one of the most popular machine learning frameworks among data scientists, with this step-by-step tutorial in Python.

XGBoost is one of the most popular machine learning frameworks among data scientists. According to the Kaggle [State of Data Science Survey 2021](https://www.kaggle.com/kaggle-survey-2021), almost 50% of respondents said they used XGBoost, ranking below only TensorFlow and Sklearn. 



https://www.kaggle.com/kaggle-survey-2021



This XGBoost tutorial will introduce the key aspects of this popular Python framework, exploring how you can use it for your own machine learning projects.

## What You Will Learn in This Python XGBoost Tutorial

Throughout this tutorial, we will cover the key aspects of XGBoost, including:

- Installation
- XGBoost DMatrix class
- XGBoost regression
- Objective and loss functions in XGBoost
- Building training and evaluation loops
- Cross-validation in XGBoost
- Building an XGBoost classifier
- Changing between Sklearn and native APIs of XGBoost


Let’s get started!

## Loading and Exploring the Data

We will be working with the Diamonds dataset throughout the tutorial. It is built into the Seaborn library, or alternatively, you can also [download it from Kaggle](https://www.kaggle.com/datasets/shivam2503/diamonds). It has a nice combination of numeric and categorical features and over 50k observations that we can comfortably showcase all the advantages of XGBoost.

In [63]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import xgboost

warnings.filterwarnings("ignore")
diamonds = sns.load_dataset("diamonds")
diamonds.head()


,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [34]:
diamonds.shape

(53940, 10)

In a typical real-world project, you would want to spend a lot more time exploring the dataset and visualizing its features. But since this data comes built-in to Seaborn, it is relatively clean.

So, we will just look at the 5-number summary of the numeric and categorical features and get going. You can spend a few moments to familiarize yourself with the dataset.

In [35]:
diamonds.describe()

,carat,depth,table,price,x,y,z
count,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000
mean,0.797940,61.749405,57.457184,3932.799722,5.731157,5.734526,3.538734
std,0.474011,1.432621,2.234491,3989.439738,1.121761,1.142135,0.705699
min,0.200000,43.000000,43.000000,326.000000,0.000000,0.000000,0.000000
25%,0.400000,61.000000,56.000000,950.000000,4.710000,4.720000,2.910000
50%,0.700000,61.800000,57.000000,2401.000000,5.700000,5.710000,3.530000
75%,1.040000,62.500000,59.000000,5324.250000,6.540000,6.540000,4.040000
max,5.010000,79.000000,95.000000,18823.000000,10.740000,58.900000,31.800000


In [36]:
diamonds.describe(exclude=np.number)

,cut,color,clarity
count,53940,53940,53940
unique,5,7,8
top,Ideal,G,SI1
freq,21551,11292,13065


## How to Build an XGBoost DMatrix

After you are done with exploration, the first step in any project is framing the machine learning problem and extracting the feature and target arrays based on the dataset.

In this tutorial, we will first try to predict diamond prices using their physical measurements, so our target will be the price column.

So, we are isolating the features into X and the target into y:

In [37]:
from sklearn.model_selection import train_test_split

# Extract feature and target arrays
X, y = diamonds.drop('price', axis=1), diamonds[['price']]

The dataset has three categorical columns. Normally, you would encode them with ordinal or one-hot encoding, but XGBoost has the ability to internally deal with categoricals.

The way to enable this feature is to cast the categorical columns into Pandas `category` data type (by default, they are treated as text columns):

In [38]:
X.dtypes

carat       float64
cut        category
color      category
clarity    category
depth       float64
table       float64
x           float64
y           float64
z           float64
dtype: object

In [39]:
# Extract text features
cats = X.select_dtypes(exclude=np.number).columns.tolist()

# Convert to Pandas category
for col in cats:
   print(col)
   X[col] = X[col].astype('category')

cut
color
clarity


Now, when you print the `dtypes` attribute, you'll see that we have three `category` features:

In [40]:
X.dtypes

carat       float64
cut        category
color      category
clarity    category
depth       float64
table       float64
x           float64
y           float64
z           float64
dtype: object

Let’s split the data into train, and test sets (0.25 test size):

In [41]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

Now, the important part: XGBoost comes with its own class for storing datasets called DMatrix. It is a highly optimized class for memory and speed. That's why converting datasets into this format is a requirement for the native XGBoost API:

In [42]:
import xgboost as xgb

# Create regression matrices
dtrain_reg = xgb.DMatrix(X_train, y_train, enable_categorical=True)
dtest_reg = xgb.DMatrix(X_test, y_test, enable_categorical=True)

The class accepts both the training features and the labels. To enable automatic encoding of Pandas category columns, we also set `enable_categorical` to True.

**Note**:

Why are we going with the native API of XGBoost, rather than its Scikit-learn API? While it might be more comfortable to use the Sklearn API at first, later, you’ll realize that the native API of XGBoost contains some excellent features that the former doesn’t support. So, better get used to it from the beginning. However, there is a section at the end where we show how to switch between APIs in a single line of code even after you have trained models.

## Python XGBoost Regression

After building the DMatrices, you should choose a value for the `objective` parameter. It tells XGBoost the machine learning problem you are trying to solve and what metrics or loss functions to use to solve that problem.

For example, to predict diamond prices, which is a regression problem, you can use the common `reg:squarederror` objective. Usually, the name of the objective also contains the name of the loss function for the problem. For regression, it is common to use Root Mean Squared Error, which minimizes the square root of the squared sum of the differences between actual and predicted values. Here is how the metric would look like when implemented in NumPy:

```python
import numpy as np

mse = np.mean((actual - predicted) ** 2)
rmse = np.sqrt(mse)
```

We’ll learn classification objectives later in the tutorial.

A note on the difference between a loss function and a performance metric: A loss function is used by machine learning models to minimize the _differences_ between the actual (ground truth) values and model predictions. On the other hand, a **metric** (or metrics) is chosen by the machine learning engineer to measure the _similarity_ between ground truth and model predictions.

In short, a loss function should be minimized while a metric should be maximized. A loss function is used during training to guide the model on where to improve. A metric is used during evaluation to measure overall performance.

### Training

The chosen objective function and any other hyperparameters of XGBoost should be specified in a dictionary, which by convention should be called params:

In [44]:
# Define hyperparameters
params = {"objective": "reg:squarederror", "tree_method": "hist"}

# In case you have access to a GPU
# params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}

Now, we set another parameter called `num_boost_round`, which stands for _number_ _of boosting rounds_. Internally, XGBoost minimizes the loss function RMSE in small incremental rounds (more on this later). This parameter specifies the amount of those rounds.

The ideal number of rounds is found through hyperparameter tuning. For now, we will just set it to 100:

In [45]:
n = 100
model = xgb.train(
   params=params,
   dtrain=dtrain_reg,
   num_boost_round=n,
)

When XGBoost runs on a GPU, it is blazing fast. If you didn’t receive any errors from the above code, the training was successful!

### Evaluation

During the boosting rounds, the model object has learned all the patterns of the training set it possibly can. Now, we must measure its performance by testing it on unseen data. That's where our `dtest_reg` DMatrix comes into play:

In [46]:
from sklearn.metrics import mean_squared_error

preds = model.predict(dtest_reg)

This step of the process is called model evaluation (or inference). Once you generate predictions with predict, you pass them inside `mean_squared_error` function of Sklearn to compare against `y_test`:

In [47]:
rmse = mean_squared_error(y_test, preds, squared=False)

print(f"RMSE of the base model: {rmse:.3f}")

RMSE of the base model: 545.388


We’ve got a base score ~543$, which was the performance of a base model with default parameters. There are two ways we can improve it— by performing cross-validation and hyperparameter tuning. But before that, let’s see a quicker way of evaluating XGBoost models.

## Using Validation Sets During Training

Training a machine learning model is like launching a rocket into space. You can control everything about the model up to the launch, but once it does, all you can do is stand by and wait for it to finish.

But the problem with our current training process is that we can’t even watch where the model is going. To solve this, we will use evaluation arrays that allow us to see model performance as it gets improved incrementally across boosting rounds.

First, let’s set up the parameters again:

In [48]:
params = {"objective": "reg:squarederror", "tree_method": "hist"}
n = 100

Next, we create a list of two tuples that each contain two elements. The first element is the array for the model to evaluate, and the second is the array’s name.

In [49]:
evals = [(dtrain_reg, "train"), (dtest_reg, "validation")]

When we pass this array to the `evals` parameter of `xgb.train`, we will see the model performance after each boosting round:

In [50]:
model = xgb.train(
   params=params,
   dtrain=dtrain_reg,
   num_boost_round=n,
   evals=evals,
)

[0]	train-rmse:3985.31595	validation-rmse:3930.87087
[1]	train-rmse:2849.92126	validation-rmse:2812.52945
[2]	train-rmse:2061.76472	validation-rmse:2034.91266
[3]	train-rmse:1521.58802	validation-rmse:1509.03801
[4]	train-rmse:1158.20689	validation-rmse:1155.77477
[5]	train-rmse:918.95666	validation-rmse:922.81058
[6]	train-rmse:765.71970	validation-rmse:778.96367
[7]	train-rmse:671.73734	validation-rmse:692.56259
[8]	train-rmse:612.92636	validation-rmse:638.83852
[9]	train-rmse:578.33182	validation-rmse:608.53984
[10]	train-rmse:557.19710	validation-rmse:591.03042
[11]	train-rmse:542.58916	validation-rmse:578.99646
[12]	train-rmse:534.88302	validation-rmse:573.25964
[13]	train-rmse:527.20423	validation-rmse:566.28647
[14]	train-rmse:520.90582	validation-rmse:561.97563
[15]	train-rmse:515.69808	validation-rmse:558.92935
[16]	train-rmse:512.45290	validation-rmse:557.84167
[17]	train-rmse:507.50759	validation-rmse:556.68519
[18]	train-rmse:504.04144	validation-rmse:553.56230
[19]	train-r

You can see how the model minimizes the score from a whopping ~3931\$ to just 543\$.

What’s best is that we can see the model’s performance on both our training and validation sets. Usually, the training loss will be lower than validation since the model has already seen the former.

In real-world projects, you usually train for thousands of boosting rounds, which means that many rows of output. To reduce them, you can use the `verbose_eval` parameter, which forces XGBoost to print performance updates every `vebose_eval` rounds:

In [51]:
params = {"objective": "reg:squarederror", "tree_method": "hist"}
n = 100

evals = [(dtest_reg, "validation"), (dtrain_reg, "train")]


model = xgb.train(
   params=params,
   dtrain=dtrain_reg,
   num_boost_round=n,
   evals=evals,
   verbose_eval=10 # Every ten rounds
)

[0]	validation-rmse:3930.87087	train-rmse:3985.31595
[10]	validation-rmse:591.03042	train-rmse:557.19710
[20]	validation-rmse:550.76666	train-rmse:495.31647
[30]	validation-rmse:547.16647	train-rmse:467.13670
[40]	validation-rmse:544.10422	train-rmse:447.26879
[50]	validation-rmse:543.97371	train-rmse:432.51681
[60]	validation-rmse:544.77874	train-rmse:420.72943
[70]	validation-rmse:544.77491	train-rmse:408.72053
[80]	validation-rmse:544.33808	train-rmse:395.88816
[90]	validation-rmse:545.99682	train-rmse:383.62262
[99]	validation-rmse:545.38842	train-rmse:378.37454


## XGBoost Early Stopping

By now, you must have realized how important boosting rounds are. Generally, the more rounds there are, the more XGBoost tries to minimize the loss. But this doesn’t mean the loss will always go down. Let’s try with 5000 boosting rounds with the verbosity of 500:

In [52]:
params = {"objective": "reg:squarederror", "tree_method": "hist"}
n = 5000

evals = [(dtest_reg, "validation"), (dtrain_reg, "train")]


model = xgb.train(
   params=params,
   dtrain=dtrain_reg,
   num_boost_round=n,
   evals=evals,
   verbose_eval=250
)

[0]	validation-rmse:3930.87087	train-rmse:3985.31595
[250]	validation-rmse:557.68923	train-rmse:282.14076
[500]	validation-rmse:567.09440	train-rmse:202.29270
[750]	validation-rmse:571.09355	train-rmse:153.40105
[1000]	validation-rmse:573.92496	train-rmse:124.11251
[1250]	validation-rmse:575.80595	train-rmse:102.84756
[1500]	validation-rmse:577.05346	train-rmse:86.53817
[1750]	validation-rmse:578.72790	train-rmse:74.95409
[2000]	validation-rmse:579.29924	train-rmse:64.88654
[2250]	validation-rmse:580.10885	train-rmse:56.54860
[2500]	validation-rmse:580.64166	train-rmse:49.60797
[2750]	validation-rmse:581.06804	train-rmse:43.65126
[3000]	validation-rmse:581.33497	train-rmse:39.07501
[3250]	validation-rmse:581.68045	train-rmse:35.19566
[3500]	validation-rmse:581.94197	train-rmse:31.53707
[3750]	validation-rmse:582.01889	train-rmse:28.69983
[4000]	validation-rmse:582.09487	train-rmse:26.22932
[4250]	validation-rmse:582.27258	train-rmse:24.02182
[4500]	validation-rmse:582.35214	train-rmse:

We get the lowest loss before round 500. After that, even though training loss keeps going down, the validation loss (the one we care about) keeps increasing.

When given an unnecessary number of boosting rounds, XGBoost starts to overfit and memorize the dataset. This, in turn, leads to validation performance drop because the model is memorizing instead of generalizing.

Remember, we want the **golden middle**: a model that learned just enough patterns in training that it gives the highest performance on the validation set. So, how do we find the perfect number of boosting rounds, then?

We will use a technique called **early stopping**. Early stopping forces XGBoost to watch the validation loss, and if it stops improving for a specified number of rounds, it automatically stops training.

This means we can set as high a number of boosting rounds as long as we set a sensible number of early stopping rounds.

For example, let’s use 10000 boosting rounds and set the `early_stopping_rounds` parameter to 50. This way, XGBoost will automatically stop the training if validation loss doesn't improve for 50 consecutive rounds.

In [53]:
n = 5000

model = xgb.train(
   params=params,
   dtrain=dtrain_reg,
   num_boost_round=n,
   evals=evals,
   verbose_eval=100,
   # Activate early stopping
   early_stopping_rounds=50
)

[0]	validation-rmse:3930.87087	train-rmse:3985.31595
[100]	validation-rmse:545.56388	train-rmse:377.95218
[200]	validation-rmse:554.85898	train-rmse:309.24955
[300]	validation-rmse:562.06936	train-rmse:260.56315
[400]	validation-rmse:564.23042	train-rmse:228.55880
[500]	validation-rmse:567.09440	train-rmse:202.29270
[600]	validation-rmse:568.63118	train-rmse:179.81322
[700]	validation-rmse:569.82263	train-rmse:162.18310
[800]	validation-rmse:571.55832	train-rmse:146.51777
[900]	validation-rmse:573.03163	train-rmse:134.76290
[1000]	validation-rmse:573.92496	train-rmse:124.11251
[1100]	validation-rmse:574.99063	train-rmse:115.12676
[1200]	validation-rmse:575.17781	train-rmse:106.35001
[1300]	validation-rmse:576.22607	train-rmse:98.92870
[1400]	validation-rmse:576.52550	train-rmse:92.87394
[1500]	validation-rmse:577.05346	train-rmse:86.53817
[1600]	validation-rmse:577.67782	train-rmse:81.69339
[1700]	validation-rmse:578.43809	train-rmse:77.47407
[1800]	validation-rmse:578.93009	train-rmse

As you can see, the training stopped after the 167th round because the loss stopped improving for 50 rounds before that.

## XGBoost Cross-Validation

At the beginning of the tutorial, we set aside 25% of the dataset for testing. The test set would allow us to simulate the conditions of a model in production, where it must generate predictions for unseen data.

But only a single test set would not be enough to measure how a model would perform in production accurately. For example, if we perform hyperparameter tuning using only a single training and a single test set, knowledge about the test set would still “leak out.” How?

Since we try to find the best value of a hyperparameter by comparing the validation performance of the model on the test set, we will end up with a model that is configured to perform well _only_ on that particular test set. Instead, we want a model that performs well across the board — on any test set we throw at it.

A possible workaround is splitting the data into three sets. The model trains on the first set, the second set is used for evaluation and hyperparameter tuning, and the third is the final one we test the model before production.

But when data is limited, splitting data into three sets will make the training set sparse, which hurts model performance.

The solution to all these problems is cross-validation. In cross-validation, we still have two sets: training and testing.

While the test set waits in the corner, we split the training into 3, 5, 7, or _k_ splits or folds. Then, we train the model _k_ times. Each time, we use _k-1_ parts for training and the final _k_th part for validation. This process is called k-fold cross-validation:



Source: https://scikit-learn.org/stable/modules/cross_validation.html

Above is a visual depiction of a 5-fold cross-validation. After all folds are done, we can take the mean of the scores as the final, most realistic performance of the model.

Let’s perform this process in code using the `cv` function of XGB:

In [54]:
params = {"objective": "reg:squarederror", "tree_method": "hist"}
n = 1000

results = xgb.cv(
   params, dtrain_reg,
   num_boost_round=n,
   nfold=5,
   early_stopping_rounds=20
)

The only difference with the train function is adding the `nfold` parameter to specify the number of splits. The results object is now a DataFrame containing each fold's results:

In [55]:
results.head()

,train-rmse-mean,train-rmse-std,test-rmse-mean,test-rmse-std
0,3985.648654,10.343596,3986.913623,41.642778
1,2848.365726,8.014086,2851.020437,28.028733
2,2063.401458,4.637773,2068.629977,19.969459
3,1521.493751,3.874078,1530.496272,13.592330
4,1156.827103,2.991735,1170.413316,11.695597


It has the same number of rows as the number of boosting rounds. Each row is the average of all splits for that round. So, to find the best score, we take the minimum of the `test-rmse-mean` column:

In [56]:
best_rmse = results['test-rmse-mean'].min()
best_rmse

550.7196748119261

Note that this method of cross-validation is used to see the true performance of the model. Once satisfied with its score, you must retrain it on the full data before deployment.

## XGBoost Classification

Building an XGBoost classifier is as easy as changing the objective function; the rest can stay the same.

The two most popular classification objectives are:

- `binary:logistic` - binary classification (the target contains only two classes, i.e., cat or dog)
- `multi:softprob` - multi-class classification (more than two classes in the target, i.e., apple/orange/banana)


Performing binary and multi-class classification in XGBoost is almost identical, so we will go with the latter. Let’s prepare the data for the task first.

We want to predict the cut quality of diamonds given their price and their physical measurements. So, we will build the feature/target arrays accordingly:

In [57]:
from sklearn.preprocessing import OrdinalEncoder

X, y = diamonds.drop("cut", axis=1), diamonds[['cut']]

# Encode y to numeric
y_encoded = OrdinalEncoder().fit_transform(y)

# Extract text features
cats = X.select_dtypes(exclude=np.number).columns.tolist()

# Convert to pd.Categorical
for col in cats:
   X[col] = X[col].astype('category')

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, random_state=1, stratify=y_encoded)

The only difference is that since XGBoost only accepts numbers in the target, we are encoding the text classes in the target with `OrdinalEncoder` of Sklearn.

Now, we build the DMatrices…

In [58]:
# Create classification matrices
dtrain_clf = xgb.DMatrix(X_train, y_train, enable_categorical=True)
dtest_clf = xgb.DMatrix(X_test, y_test, enable_categorical=True)

…and set the objective to `multi:softprob`. This objective also requires the number of classes to be set by us:

In [59]:
params = {"objective": "multi:softprob", "tree_method": "hist", "num_class": 5}
n = 1000

results = xgb.cv(
   params, dtrain_clf,
   num_boost_round=n,
   nfold=5,
   metrics=["mlogloss", "auc", "merror"],
)

During cross-validation, we are asking XGBoost to watch three classification metrics which report model performance from three different angles. Here is the result:

In [60]:
results.keys()

Index(['train-mlogloss-mean', 'train-mlogloss-std', 'train-auc-mean',
       'train-auc-std', 'train-merror-mean', 'train-merror-std',
       'test-mlogloss-mean', 'test-mlogloss-std', 'test-auc-mean',
       'test-auc-std', 'test-merror-mean', 'test-merror-std'],
      dtype='object')

To see the best AUC score, we take the maximum of test-auc-mean column:

In [61]:
results['test-auc-mean'].max()

0.9403142623248778

Even the default configuration gave us 94% performance, which is great.

## XGBoost Native vs. XGBoost Sklearn

So far, we have been using the native XGBoost API, but its Sklearn API is pretty popular as well.

Sklearn is a vast framework with many machine learning algorithms and utilities and has an API syntax loved by almost everyone. Therefore, XGBoost also offers XGBClassifier and XGBRegressor classes so that they can be integrated into the Sklearn ecosystem (at the loss of some of the functionality).

If you want to only use the Scikit-learn API whenever possible and only switch to native when you need access to extra functionality, there is a way.

After training the XGBoost classifier or regressor, you can convert it using the `get_booster` method:

In [62]:
import xgboost as xgb

# Train a model using the scikit-learn API
xgb_classifier = xgb.XGBClassifier(n_estimators=100, objective='binary:logistic', tree_method='hist', eta=0.1, max_depth=3, enable_categorical=True)
xgb_classifier.fit(X_train, y_train)

# Convert the model to a native API model
model = xgb_classifier.get_booster()

The model object will behave in the exact same way we've seen throughout this tutorial.

## Conclusion

We’ve covered a lot of important topics in this XGBoost tutorial, but there are still so many things to learn.

You can check out the[ XGBoost parameters page](https://xgboost.readthedocs.io/en/stable/parameter.html), which teaches you how to configure the parameters to squeeze out every last performance from your models.

If you are looking for a comprehensive, all-in-one resource to learn the library, check out our [Extreme Gradient Boosting With XGBoost course](https://www.datacamp.com/courses/extreme-gradient-boosting-with-xgboost).